# Report

## Region Growing Algorithm

### Region Merging
In both algorithms, seeded and unseeded region growing too many regions are received due to intensity differences in
background as well as in cell nuclei. To solve this over-segmentation a region merging algorithm was implemented.
The received regions are merged by two criteria: similarity of intensity values and region size.
